In [128]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_type: str
    kaggle_dataset_owner: str
    kaggle_dataset_name: str
    local_data_file: Path
    unzip_dir: Path


In [129]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [130]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH,) :
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_type = config.source_type,
            kaggle_dataset_owner = config.kaggle_dataset_owner,
            kaggle_dataset_name = config.kaggle_dataset_name,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config    

In [131]:
import os
import zipfile
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size
import shutil

In [132]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self) -> str:
        # Authenticate with Kaggle API
        api = KaggleApi()
        api.authenticate()
        logger.info(f"Kaggle API Authenticated")
        
        try: 
            dataset_url = f"{self.config.kaggle_dataset_owner}/{self.config.kaggle_dataset_name}"
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")
            api.dataset_download_files(dataset_url, path=zip_download_dir, unzip=False)
            logger.info(f"Downloaded dataset from kaggle to {zip_download_dir}")
            #logger.info(f"Downloaded dataset from kaggle to {self.config.root_dir}")
        
        except Exception as e:
            raise e
      
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        zip_file_path = os.path.join(self.config.local_data_file, 'chest-ctscan-images.zip')
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)  

        
        logger.info(f"Extracted dataset to {unzip_path}")

In [133]:
try:
    config_manager = ConfigurationManager()
    data_ingestion_config = config_manager.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-08-21 23:59:32,273: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-08-21 23:59:32,274: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-21 23:59:32,275: INFO: common: created directory at: artifacts]
[2024-08-21 23:59:32,275: INFO: common: created directory at: artifacts/data_ingestion]
[2024-08-21 23:59:32,278: INFO: 782049457: Kaggle API Authenticated]
[2024-08-21 23:59:32,279: INFO: 782049457: Downloading data from mohamedhanyyy/chest-ctscan-images into file artifacts/data_ingestion/]
Dataset URL: https://www.kaggle.com/datasets/mohamedhanyyy/chest-ctscan-images
[2024-08-21 23:59:37,021: INFO: 782049457: Downloaded dataset from kaggle to artifacts/data_ingestion/]
[2024-08-21 23:59:37,656: INFO: 782049457: Extracted dataset to artifacts/data_ingestion]
